In [59]:
import yaml
import wandb
import time

import pandas as pd
with open('config/sweep.yaml') as f:
    sweep_config = yaml.load(f, Loader=yaml.FullLoader)
display(sweep_config)


{'program': 'model.train',
 'method': 'bayes',
 'metric': {'name': 'rmse', 'goal': 'minimize'},
 'parameters': {'learning_rate': {'min': 0.001, 'max': 0.1},
  'optimizer': {'values': ['adam', 'sgd']},
  'epoch': {'min': 5, 'max': 20}}}

In [60]:
with open('config/config.yaml') as f:
    default_config = yaml.load(f, Loader=yaml.FullLoader)
#display(default_config)

In [61]:
import easydict
 
args = easydict.EasyDict(default_config)

print(type(args.WEIGHT_DECAY))
print(args.WEIGHT_DECAY)
args.WEIGHT_DECAY = float(args.WEIGHT_DECAY)
print(type(args.WEIGHT_DECAY))
print(args.WEIGHT_DECAY)
################# 돌릴 모델 바꾸려면 여기서
args.MODEL = "FM"

<class 'str'>
1e-06
<class 'float'>
1e-06


In [62]:
"""
config={
            "epochs": args.EPOCHS,
            "batch_size": args.BATCH_SIZE,
            "lr": args.LR,
            "emb_dim": 16
            }

            """

'\nconfig={\n            "epochs": args.EPOCHS,\n            "batch_size": args.BATCH_SIZE,\n            "lr": args.LR,\n            "emb_dim": 16\n            }\n\n            '

In [63]:
"""
wandb.init(
        project="seongjae_exp_sweep", 
        entity="boostcamp_l1_recsys05",
        name=f"experiment_{args.MODEL}", 
        # Track hyperparameters and run metadata
        config=config)"""
        

'\nwandb.init(\n        project="seongjae_exp_sweep", \n        entity="boostcamp_l1_recsys05",\n        name=f"experiment_{args.MODEL}", \n        # Track hyperparameters and run metadata\n        config=config)'

MAIN.PY  모델 도입부

In [64]:
from src import (CNN_FM, DeepCoNN, DeepCrossNetworkModel,
                 FactorizationMachineModel,
                 FieldAwareFactorizationMachineModel,
                 NeuralCollaborativeFiltering, WideAndDeepModel,
                 seed_everything)
from src.data import (context_data_load, context_data_loader,
                      context_data_split, dl_data_load, dl_data_loader,
                      dl_data_split, image_data_load, image_data_loader,
                      image_data_split, text_data_load, text_data_loader,
                      text_data_split)

In [65]:
######################## DATA LOAD
print(f'--------------- {args.MODEL} Load Data ---------------')
if args.MODEL in ('FM', 'FFM'):
    data = context_data_load(args)
elif args.MODEL in ('NCF', 'WDN', 'DCN'):
    data = dl_data_load(args)
elif args.MODEL == 'CNN_FM':
    data = image_data_load(args)
elif args.MODEL == 'DeepCoNN':
    import nltk
    nltk.download('punkt')
    data = text_data_load(args)
else:
    pass

######################## Train/Valid Split
print(f'--------------- {args.MODEL} Train/Valid Split ---------------')
if args.MODEL in ('FM', 'FFM'):
    data = context_data_split(args, data)
    data = context_data_loader(args, data)
elif args.MODEL in ('NCF', 'WDN', 'DCN'):
    data = dl_data_split(args, data)
    data = dl_data_loader(args, data)
elif args.MODEL=='CNN_FM':
    data = image_data_split(args, data)
    data = image_data_loader(args, data)
elif args.MODEL=='DeepCoNN':
    data = text_data_split(args, data)
    data = text_data_loader(args, data)
else:
    pass
data

--------------- FM Load Data ---------------


In [ ]:
def sweep(args, data):
    ######################## Model
    wandb.init(project="sweep_tmp")
    print(f'--------------- INIT {args.MODEL} ---------------')
    if args.MODEL=='FM':
        print("wandb.config.emb_dim = ", wandb.config.emb_dim)
        print(type(wandb.config.emb_dim))
        args.FM_EMBED_DIM = wandb.config.emb_dim
        model = FactorizationMachineModel(args, data)
    elif args.MODEL=='FFM':
        args.FFM_EMBED_DIM = wandb.config.emb_dim
        model = FieldAwareFactorizationMachineModel(args, data)
    elif args.MODEL=='NCF':
        args.NCF_EMBED_DIM = wandb.config.emb_dim
        model = NeuralCollaborativeFiltering(args, data)
    elif args.MODEL=='WDN':
        args.WDN_EMBED_DIM = wandb.config.emb_dim
        model = WideAndDeepModel(args, data)
    elif args.MODEL=='DCN':
        args.DCN_EMBED_DIM = wandb.config.emb_dim
        model = DeepCrossNetworkModel(args, data)
    elif args.MODEL=='CNN_FM':
        args.CNN_FM_EMBED_DIM = wandb.config.emb_dim
        model = CNN_FM(args, data)
    elif args.MODEL=='DeepCoNN':
        args.DEEPCONN_EMBED_DIM = wandb.config.emb_dim
        model = DeepCoNN(args, data)
    else:
        pass

    #wandb.config.update(args)
    # wandb.watch(model)
    ######################## TRAIN
    print(f'--------------- {args.MODEL} TRAINING ---------------')
    rmse = model.train()
    
    ######################## INFERENCE
    print(f'--------------- {args.MODEL} PREDICT ---------------')
    if args.MODEL in ('FM', 'FFM', 'NCF', 'WDN', 'DCN'):
        predicts = model.predict(data['test_dataloader'])
    elif args.MODEL=='CNN_FM':
        predicts  = model.predict(data['test_dataloader'])
    elif args.MODEL=='DeepCoNN':
        predicts  = model.predict(data['test_dataloader'])
    else:
        pass
    
    ######################## PREDICT GET IN RANGE
    def adjust_predict(y):
        if y < 1.0:
            return 1.0
        elif y > 10.0:
            return 10.0
        return y
    
    predicts = list(map(adjust_predict, predicts))

    ######################## SAVE PREDICT
    print(f'--------------- SAVE {args.MODEL} PREDICT ---------------')
    submission = pd.read_csv(args.DATA_PATH + 'sample_submission.csv')
    if args.MODEL in ('FM', 'FFM', 'NCF', 'WDN', 'DCN', 'CNN_FM', 'DeepCoNN'):
        submission['rating'] = predicts
    else:
        pass
    train1 = pd.read_csv('./data/train_ratings.csv')

    count=train1.groupby("user_id").size()
    dfcount = pd.DataFrame(count, columns=["count"])
    submission=pd.merge(submission,dfcount, how='left', on='user_id')
    submission['count'] = submission['count'].fillna(0)
    submission.set_index("user_id",inplace = True)

    for row in submission.itertuples():
        if row[3] == 0 :
            submission.at[row[0],"rating"] = 7

    submission = submission.reset_index()
    submission = submission.drop(['count'], axis=1)


    now = time.localtime()
    now_date = time.strftime('%Y%m%d', now)
    now_hour = time.strftime('%X', now)
    save_time = now_date + '_' + now_hour.replace(':', '')
    submission.to_csv('submit/{}_{}_{}.csv'.format(save_time, args.MODEL, round(rmse, 5), index=False))
    

In [ ]:
sweep_configuration = {
    'method': 'bayes',
    'name': 'sweep',
    'metric': {'goal': 'minimize', 'name': 'rmse'}, 
    'parameters':{
        'emb_dim':{'max': 32, 'min': 8  }
        }}



In [ ]:
config={
            "epochs": args.EPOCHS,
            "batch_size": args.BATCH_SIZE,
            "lr": args.LR,
            "emb_dim": 16
            }



In [ ]:
wandb.login()

True

In [ ]:
wandb.init(project="sweep_tmp",
        config=config)
wandb.config.emb_dim
        

Problem at: /tmp/ipykernel_74153/933464639.py 1 <module>


Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/wandb_init.py", line 1078, in init
    run = wi.init()
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/wandb_init.py", line 574, in init
    manager._inform_init(settings=self.settings, run_id=self.settings.run_id)
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/wandb_manager.py", line 170, in _inform_init
    svc_iface._svc_inform_init(settings=settings, run_id=run_id)
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/service/service_sock.py", line 38, in _svc_inform_init
    self._sock_client.send(inform_init=inform_init)
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/lib/sock_client.py", line 211, in send
    self.send_server_request(server_req)
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/lib/sock_client.py", line 155, in send_server_request
    self._send_message(msg)
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/lib/sock_client.py",

Exception: problem

In [ ]:
wandb.config.emb_dim
args.EPOCHS = 1

Error: You must call wandb.init() before wandb.config.emb_dim

In [ ]:
sweep_id = wandb.sweep(sweep=sweep_configuration,project='sweep_tmp')
wandb.agent(sweep_id, function=sweep(args,data),count =2)

Create sweep with ID: jrpchz8d
Sweep URL: https://wandb.ai/boostcamp_l1_recsys05/seongjae_exp_sweep/sweeps/jrpchz8d


Problem at: /tmp/ipykernel_74153/2099338128.py 3 sweep


Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/wandb_init.py", line 1078, in init
    run = wi.init()
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/wandb_init.py", line 574, in init
    manager._inform_init(settings=self.settings, run_id=self.settings.run_id)
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/wandb_manager.py", line 170, in _inform_init
    svc_iface._svc_inform_init(settings=settings, run_id=run_id)
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/service/service_sock.py", line 38, in _svc_inform_init
    self._sock_client.send(inform_init=inform_init)
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/lib/sock_client.py", line 211, in send
    self.send_server_request(server_req)
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/lib/sock_client.py", line 155, in send_server_request
    self._send_message(msg)
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/lib/sock_client.py",

Exception: problem